In [72]:
import sys
from bs4 import BeautifulSoup
# from selenium import webdriver
# from selenium.common.exceptions import TimeoutException
import time
import requests
import random
import pandas as pd
from datetime import date, datetime, timedelta
import traceback
import csv
import math

In [104]:
year_list = [x for x in range(2010,2019)]
page_list = [x for x in range(1,130)]

# 2008 56897 57
# 2009 37884 38
# 2010 80943 81
# 2011 73210 74
# 2012 63400 64
# 2013 43702 44
# 2014 70651 71
# 2015 88924 89
# 2016 125674 126
# 2017 86535 87
# 2018 34866 35


for year in year_list:

    search_url = \
            'http://visadoor.com/greencards/index?company=&job=&country=&state=&year='+\
                str(year)+'&submit=Search'
    search_page = requests.get(search_url)
    soup = BeautifulSoup(search_page.text, "html.parser")
    number_records = soup.find(name='div', attrs={"class":"col-sm-5"}).next.next.split()[3]
    number_pages = math.ceil(int(number_records)/1000)
    print("year:", year, "number_pages:", number_pages)

    for page in page_list:
        
        if page > number_pages:
            "No records here. Skipping."
            continue

        search_url = \
            'http://visadoor.com/greencards/index?company=&job=&country=&state=&year='+\
                str(year)+'&page='+str(page)+'&submit=Search'

        search_page = requests.get(search_url, timeout=30)

        soup = BeautifulSoup(search_page.text, "html.parser")
        
        print('page:', page, 'url:', search_url)
                        
        if not soup:
            print("Nothing to scrape here.")
            break

        i=1
        for record in soup.find_all(name="td", attrs={"valign":"top"}):

            if i==1:
                id_value = record.next.next
                i+=1
                continue
            if i==2:
                decision_date = record.next
                i+=1
                continue
            if i==3:
                employer = record.next.next
                i+=1
                continue
            if i==4:
                location = record.next
                i+=1
                continue
            if i==5:
                status = record.next
                i+=1
                continue
            if i==6:
                job_title = record.next
                i+=1
                continue
            if i==7:
                wage_offer = record.next
                row = [id_value, decision_date, employer, location, status, job_title, wage_offer]
                with open(str(year)+"_greencard_salaries.csv", 'a') as myfile:
                    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                    wr.writerow(row)    
                i=1
                continue

In [99]:
data_2008 = pd.read_csv('2008_greencard_salaries.csv',header=None)
data_2009 = pd.read_csv('2009_greencard_salaries.csv',header=None)
data_2010 = pd.read_csv('2010_greencard_salaries.csv',header=None)
data_2011 = pd.read_csv('2011_greencard_salaries.csv',header=None)
data_2012 = pd.read_csv('2012_greencard_salaries.csv',header=None)
data_2013 = pd.read_csv('2013_greencard_salaries.csv',header=None)
data_2014 = pd.read_csv('2014_greencard_salaries.csv',header=None)
data_2015 = pd.read_csv('2015_greencard_salaries.csv',header=None)
data_2016 = pd.read_csv('2016_greencard_salaries.csv',header=None)
data_2017 = pd.read_csv('2017_greencard_salaries.csv',header=None)
data_2018 = pd.read_csv('2018_greencard_salaries.csv',header=None)

data_2008.columns = ['case_id','decision_date','company','location','status','job_title','salary']
data_2009.columns = ['case_id','decision_date','company','location','status','job_title','salary']
data_2010.columns = ['case_id','decision_date','company','location','status','job_title','salary']
data_2011.columns = ['case_id','decision_date','company','location','status','job_title','salary']
data_2012.columns = ['case_id','decision_date','company','location','status','job_title','salary']
data_2013.columns = ['case_id','decision_date','company','location','status','job_title','salary']
data_2014.columns = ['case_id','decision_date','company','location','status','job_title','salary']
data_2015.columns = ['case_id','decision_date','company','location','status','job_title','salary']
data_2016.columns = ['case_id','decision_date','company','location','status','job_title','salary']
data_2017.columns = ['case_id','decision_date','company','location','status','job_title','salary']
data_2018.columns = ['case_id','decision_date','company','location','status','job_title','salary']

In [100]:
data = pd.concat([data_2008,
                  data_2009,
                  data_2010,
                  data_2011,
                  data_2012,
                  data_2013,
                  data_2014,
                  data_2015,
                  data_2016,
                  data_2017,
                  data_2018,
                 ],axis=0)

In [101]:
data.to_csv('greencard_salaries.csv')

In [103]:
data.head()

,case_id,decision_date,company,location,status,job_title,salary
0,A-06319-80488,9/30/08,"Safeway, Inc.","Woodbridge, VA",Certified,"Bakers, Manufacturing, LEVEL III",11.94/HR
1,C-08091-37317,9/30/08,"Restaurant Development Services, Inc.","Los Angeles, CA",Denied,"Architects, LEVEL I",51888/YR
2,A-08133-50903,9/30/08,"Cgi Federal, Inc.","Fairfax, VA",Certified,"Computer Programmer, LEVEL I",72000/YR
3,A-06219-46761,9/30/08,"Papco, Inc.","Virginia Beach, VA",Denied,"Senior Financial Analyst, LEVEL II",50000/YR
4,C-08142-54294,9/30/08,"Blue Ambrosia, Inc.","Los Angeles, CA",Denied,"Marketing Managers, LEVEL I",31.84/HR
